In [38]:
from bs4 import BeautifulSoup
import requests
import lxml
import pandas as pd
import functools as ft


In [10]:
table_names = {'Premier League':f'results{datetime.datetime.now().year - 1}-{datetime.datetime.now().year}91_overall',
                'La Liga':f'results{datetime.datetime.now().year - 1}-{datetime.datetime.now().year}121_overall',
                'Serie A':f'results{datetime.datetime.now().year - 1}-{datetime.datetime.now().year}111_overall',
                'Ligue 1':f'results{datetime.datetime.now().year - 1}-{datetime.datetime.now().year}131_overall',
                'Bundesliga':f'results{datetime.datetime.now().year - 1}-{datetime.datetime.now().year}201_overall'}

In [41]:
%run ../../python_files/get_links.py
%run ../../python_files/input_data.py

In [12]:
league_choice = 'Premier League'
team_choice = 'Manchester City'

In [13]:
league_links = get_league_links(league_choice)

In [15]:
league_links

({'Arsenal': 'https://fbref.com/en/squads/18bb7c10/Arsenal-Stats',
  'Liverpool': 'https://fbref.com/en/squads/822bd0ba/Liverpool-Stats',
  'Manchester City': 'https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats',
  'Aston Villa': 'https://fbref.com/en/squads/8602292d/Aston-Villa-Stats',
  'Tottenham Hotspur': 'https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats',
  'Manchester United': 'https://fbref.com/en/squads/19538871/Manchester-United-Stats',
  'Newcastle United': 'https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats',
  'West Ham United': 'https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats',
  'Chelsea': 'https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats',
  'Bournemouth': 'https://fbref.com/en/squads/4ba7cbea/Bournemouth-Stats',
  'Brighton and Hove Albion': 'https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats',
  'Wolverhampton Wanderers': 'https://fbref.com/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats',
  'Fulham': 'https:/

In [21]:
url = league_links[0]['Manchester City']
page = requests.get(url)
soup = BeautifulSoup(page.text, 'lxml')


In [89]:
def get_passing_combined(soup,league_id):
    
    # Retrieve the table data from html text
    table = soup.find('table', id='stats_passing_{}'.format(league_id))
    
    # Retrieve the column names from the table
    headers = get_headers(table)
    
    # Modify headers
    headers = headers[8:37]
    headers[11] = 'Attt'
    headers[14] = 'Attt'
    headers[17] = 'Attt'

    # Create DataFrame with headers
    mydata = pd.DataFrame(columns = headers)
    print(mydata)

    # Retrieve stats from the table
    overall = get_stats(table)
    
    chunks = clean_data(overall,29,False)

    mydata = input_data(chunks,mydata)
    print(mydata)
    
    # Modify data and perform calculations
    mydata = mydata.astype({'90s':'float64','Att':'int','Ast':'int','xAG':'float64'})
    mydata['Att/90'] = mydata['Att'] / mydata['90s']
    mydata['Ast/90'] = mydata['Ast'] / mydata['90s']
    
    # Create final table
    mydata_final = mydata[['Player','90s','Ast','Att/90','Ast/90']] #'Att/90','Ast/90'

    return mydata_final

In [90]:
get_passing_combined(soup,get_league_id(league_choice))

Empty DataFrame
Columns: [Player, Nation, Pos, Age, 90s, Cmp, Att, Cmp%, TotDist, PrgDist, Cmp, Attt, Cmp%, Cmp, Attt, Cmp%, Cmp, Attt, Cmp%, Ast, xAG, xA, A-xAG, KP, 1/3, PPA, CrsPA, PrgP, Matches]
Index: []

[0 rows x 29 columns]
             Player   Nation    Pos     Age   90s   Cmp   Att  Cmp% TotDist  \
0    Julián Álvarez   ar ARG  MF,FW  24-085  27.2   938  1176  79.8   13015   
1        Phil Foden  eng ENG  FW,MF  23-333  27.2  1279  1496  85.5   17266   
2             Rodri   es ESP     MF  27-308  26.7  2734  2981  91.7   47183   
3           Ederson   br BRA     GK  30-252  26.7   923  1061  87.0   22605   
4        Rúben Dias   pt POR     DF  26-347  25.4  2271  2425  93.6   39482   
5       Kyle Walker  eng ENG     DF  33-333  25.1  1759  2003  87.8   30298   
6    Erling Haaland   no NOR     FW  23-279  24.2   241   320  75.3    2692   
7    Bernardo Silva   pt POR  MF,FW  29-259  22.9  1318  1543  85.4   20427   
8     Manuel Akanji   ch SUI  DF,MF  28-281  22.1  1743  

,Player,90s,Ast,Att/90,Ast/90
0,Julián Álvarez,27.2,8,43.235294,0.294118
1,Phil Foden,27.2,7,55.000000,0.257353
2,Rodri,26.7,7,111.647940,0.262172
3,Ederson,26.7,0,39.737828,0.000000
4,Rúben Dias,25.4,0,95.472441,0.000000
5,Kyle Walker,25.1,2,79.800797,0.079681
6,Erling Haaland,24.2,5,13.223140,0.206612
7,Bernardo Silva,22.9,4,67.379913,0.174672
8,Manuel Akanji,22.1,0,84.660633,0.000000
9,Joško Gvardiol,19.9,1,76.030151,0.050251
